This cleaning file results in one combined file (poverty.csv) containing data on poverty.    

Data source: https://www.census.gov/programs-surveys/saipe/data/datasets.html

In [2]:
import os
import sys
import requests
import pandas as pd


current_dir =  os.getcwd()

grandparent_dir = os.path.dirname(os.path.dirname(current_dir))
sys.path.insert(0, grandparent_dir)

In [28]:

dataframes = []

for year in range(1998, 2021 + 1):
 
    api_url = f"https://api.census.gov/data/timeseries/poverty/saipe?get=YEAR,GEOID,SAEMHI_PT,SAEPOV0_17_PT,SAEPOVRT0_17_PT,SAEPOVALL_PT,SAEPOVRTALL_PT&for=county:*&in=state:*&time={year}"
    

    response = requests.get(api_url)
    
   
    if response.status_code == 200:
       
        data = response.json()
        columns = data[0]
        data = data[1:]
        df = pd.DataFrame(data, columns=columns)
        dataframes.append(df)
        print(f"DataFrame for year {year} created successfully.")

poverty_raw = pd.concat(dataframes, ignore_index=True)

DataFrame for year 1998 created successfully.
DataFrame for year 1999 created successfully.
DataFrame for year 2000 created successfully.
DataFrame for year 2001 created successfully.
DataFrame for year 2002 created successfully.
DataFrame for year 2003 created successfully.
DataFrame for year 2004 created successfully.
DataFrame for year 2005 created successfully.
DataFrame for year 2006 created successfully.
DataFrame for year 2007 created successfully.
DataFrame for year 2008 created successfully.
DataFrame for year 2009 created successfully.
DataFrame for year 2010 created successfully.
DataFrame for year 2011 created successfully.
DataFrame for year 2012 created successfully.
DataFrame for year 2013 created successfully.
DataFrame for year 2014 created successfully.
DataFrame for year 2015 created successfully.
DataFrame for year 2016 created successfully.
DataFrame for year 2017 created successfully.
DataFrame for year 2018 created successfully.
DataFrame for year 2019 created su

In [38]:
poverty_raw.rename(columns={'YEAR': 'Year', 'GEOID': 'GeoFIPS', 'SAEMHI_PT': 'medianHouseholdIncome',
'SAEPOV0_17_PT': 'povertyUnder18', 'SAEPOVRTALL_PT':'povertyAllpr', 'SAEPOVALL_PT':'povertyAll',
'SAEPOVRT0_17_PT': 'povertyUnder18pr'
    }, inplace=True)

columns_to_keep = ['GeoFIPS', 'Year', 'povertyAll', 'povertyAllpr', 
        'povertyUnder18', 'povertyUnder18pr', 'medianHouseholdIncome']
poverty_raw = poverty_raw.loc[:, columns_to_keep]

In [ ]:
poverty_raw.to_csv(f"{grandparent_dir}/data/raw/poverty.csv", index=False)


In [28]:
poverty = pd.read_csv(f"{grandparent_dir}/data/raw/poverty.csv")
from cities.utils.data_grabber import DataGrabber

In [8]:
data = DataGrabber()
data.get_features_wide(["gdp"])
gdp = data.wide["gdp"]

import numpy as np

In [14]:
common_fips = np.intersect1d(gdp["GeoFIPS"].unique(), poverty["GeoFIPS"].unique())

poverty = poverty[poverty["GeoFIPS"].isin(common_fips)]

poverty = poverty.merge(gdp[["GeoFIPS", "GeoName"]], on="GeoFIPS", how="left")

poverty = poverty.sort_values(by=["GeoFIPS", "GeoName"])

In [16]:
desired_order = ['GeoFIPS', 'GeoName', 'Year', 'povertyAll', 'povertyAllpr',
                  'povertyUnder18', 'povertyUnder18pr', 'medianHouseholdIncome' ]

poverty = poverty[desired_order]


In [23]:
poverty.to_csv(f"{grandparent_dir}/data/raw/poverty.csv", index=False)
